In [ ]:
from dtw import *; 
import numpy as np
from scipy.spatial.distance import cdist
import pandas as pd

In [ ]:
#one file processed at a time

import numpy as np
import pandas as pd
from dtw import *
import matplotlib.pyplot as plt
import plotly.graph_objects as go

sheet1 = "D11"
df1 = pd.read_excel(r"D:\00_1st_Year_PhD\00_Manuscript_2\Data\JSQH317\DISPERSION\Track_19.xlsx", sheet_name=sheet1)

sheet2 = "D12"
df2 = pd.read_excel(r"D:\00_1st_Year_PhD\00_Manuscript_2\Data\JSQH317\DISPERSION\Track_1249.xlsx", sheet_name=sheet2)

x_traj1 = df1["POSITION_X"].values
y_traj1 = df1["POSITION_Y"].values
z_traj1 = df1["POSITION_Z"].values

x_traj2 = df2["POSITION_X"].valuesP1
y_traj2 = df2["POSITION_Y"].values
z_traj2 = df2["POSITION_Z"].values


traj1 = np.column_stack((x_traj1, y_traj1, z_traj1))
traj2 = np.column_stack((x_traj2, y_traj2, z_traj2))
#print(traj1)
#print(traj2)

# alignment = dtw(traj1, traj2, keep_internals=True, open_end=True, 
#                  dist_method = 'euclidean',
#                  step_pattern = symmetric2)


alignment2 = dtw(traj1, traj2, keep_internals=True, open_end=True, 
                 dist_method = 'euclidean',
                 step_pattern = symmetricP1)



#norm = alignment.normalizedDistance
norm2 = alignment2.normalizedDistance
#print(alignment)

# dtw_distance = alignment.distance
# print(dtw_distance)
# print(norm)

dtw_distance2 = alignment2.distance
print(dtw_distance2)
print(norm2)

#matrix = alignment.costMatrix
#print(matrix)

#dtwPlot(alignment, type='alignment')
dtwPlot(alignment2, type='alignment')

distance_data = {"Step": [], "Distance": []}

warp_path = alignment2.index1
distances = alignment2.index2

for i, (idx1, idx2) in enumerate(zip(warp_path, distances)):
    idx1, idx2 = int(idx1), int(idx2)
    if idx1 < alignment2.costMatrix.shape[0] and idx2 < alignment2.costMatrix.shape[1]:
        distance = alignment2.costMatrix[idx1, idx2]
        distance_data["Step"].append(i + 1)
        distance_data["Distance"].append(distance)
        print(f"Step {i + 1}: Cost Matrix Value = {distance}")
    else:
        print(f"Invalid indices at Step {i + 1}")

dtw_distance = alignment2.distance
print(f"Total DTW distance: {dtw_distance}")

norm = alignment2.normalizedDistance
print(f"Normalized DTW distance: {norm}")

matrix = alignment2.costMatrix

distance_df = pd.DataFrame(distance_data)

distance_df.to_excel(r'D:\00_Manuscript\Plots\Fig_8_DTW\DTW_paths\_DTW_path.xlsx', index=False)



In [26]:
#all files processed together (per movie)

import numpy as np
import pandas as pd
from dtw import *
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import os
import re

directory = r"D:\00_1st_Year_PhD\00_Manuscript_2\Data\JSQH317\DISPERSION\DTW"
directory2 = r"D:\00_1st_Year_PhD\00_Manuscript_2\Plots\Dispersion"

files = [f for f in os.listdir(directory) if f.startswith('Track_') and f.endswith('processed.xlsx')]

results = []

for file in files:
    filepath = os.path.join(directory, file)
    
    try:
        sheet1 = "D12"
        df1 = pd.read_excel(filepath, sheet_name=sheet1)
        
        sheet2 = "D22"
        df2 = pd.read_excel(filepath, sheet_name=sheet2)
        
        x_traj1 = df1["POSITION_X"].values
        y_traj1 = df1["POSITION_Y"].values
        z_traj1 = df1["POSITION_Z"].values
        
        x_traj2 = df2["POSITION_X"].values
        y_traj2 = df2["POSITION_Y"].values
        z_traj2 = df2["POSITION_Z"].values
        
        traj1 = np.column_stack((x_traj1, y_traj1, z_traj1))
        traj2 = np.column_stack((x_traj2, y_traj2, z_traj2))
        
        alignment = dtw(traj1, traj2, keep_internals=True, open_end=True, 
                        dist_method='euclidean',
                        step_pattern=symmetricP1)
        
        norm = alignment.normalizedDistance
        dtw_distance = alignment.distance
        
        track_number = re.search(r'Track_(\d+)', file).group(1)
        
        results.append({"File": track_number, "Normalized Distance": norm, "DTW Distance": dtw_distance})
        
        print(f"Processed {file}: Normalized DTW distance = {norm}, DTW distance = {dtw_distance}")
   
        distance_data = {"Step": [], "Distance": []}
        warp_path = alignment.index1
        distances = alignment.index2
        
        for i, (idx1, idx2) in enumerate(zip(warp_path, distances)):
            idx1, idx2 = int(idx1), int(idx2)
            if idx1 < alignment.costMatrix.shape[0] and idx2 < alignment.costMatrix.shape[1]:
                distance = alignment.costMatrix[idx1, idx2]
                distance_data["Step"].append(i + 1)
                distance_data["Distance"].append(distance)
                #print(f"Step {i + 1}: Cost Matrix Value = {distance}")
            else:
                print(f"Invalid indices at Step {i + 1}")
        
        distance_df = pd.DataFrame(distance_data)
        distance_df.to_excel(os.path.join(r"D:\00_1st_Year_PhD\00_Manuscript_2\Plots\Dispersion\DTW_paths", f'{file}_DTW_path.xlsx'), index=False)
    
    except Exception as e:
        print(f"Error processing {file}: {e}")
        continue

results_df = pd.DataFrame(results)
results_df.to_excel(os.path.join(directory2, 'JSQH317_DTW_Norm_Results_D12_D22.xlsx'), index=False)
print("All results have been saved.")


Error processing Track_1050_processed.xlsx: Worksheet named 'D12' not found
Error processing Track_1051_processed.xlsx: Worksheet named 'D12' not found
Error processing Track_1078_processed.xlsx: Worksheet named 'D12' not found
Error processing Track_1145_processed.xlsx: Worksheet named 'D12' not found
Error processing Track_1168_processed.xlsx: Worksheet named 'D12' not found
Error processing Track_1170_processed.xlsx: Worksheet named 'D12' not found
Processed Track_1171_processed.xlsx: Normalized DTW distance = 227.68254058630032, DTW distance = 13205.587354005418
Processed Track_1249_processed.xlsx: Normalized DTW distance = 191.13477859250688, DTW distance = 12614.895387105455
Error processing Track_1334_processed.xlsx: Worksheet named 'D12' not found
Error processing Track_1339_processed.xlsx: Worksheet named 'D12' not found
Error processing Track_1343_processed.xlsx: Worksheet named 'D22' not found
Error processing Track_1345_processed.xlsx: Worksheet named 'D12' not found
Error 